In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#!pip install genki_widgets

In [ ]:
import os

os.chdir('../')

In [ ]:
from genki_signals.signal_sources import MouseSignalSource, Sampler

mouse_source = Sampler({'mouse_position': MouseSignalSource()}, sample_rate=100)

In [ ]:
from genki_signals.signal_system import SignalSystem
from genki_signals.signal_frontends.visualization import WidgetFrontend, Line

system = SignalSystem(mouse_source, [])

system.start()

In [ ]:
line = Line("timestamp", "mouse_position", n_visible_points = 500)

frontend = WidgetFrontend(system, widgets=[line])

In [ ]:
line.widget

In [ ]:
import genki_widgets as gw

sci_line = gw.Line(("timestamp", 0), [("mouse_position", 0), ("mouse_position", 1)])

In [ ]:
sci_line

In [ ]:
frontend.register_update_callback(id(sci_line), lambda d: sci_line.update(d.as_dict()))

In [ ]:
sci_line.n_visible_points = 500